In [ ]:
# Jupyter Notebook: analisis_recursos.ipynb

# =====================================================
# 0. Instalar dependencias
# =====================================================

%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install numpy
%pip install openpyxl

In [ ]:
# analisis_distribucion_columas.ipynb

# =====================================================
# 1. Importar librerías
# =====================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Para que los gráficos se vean en el notebook
%matplotlib inline

# Configuración estética 
sns.set(style="whitegrid", palette="muted", font_scale=1.1)

# =====================================================
# 2. Cargar el archivo en un DataFrame
# =====================================================

df_thetech = pd.read_excel("../Recursos/OutputEditado.xlsx")

# Combinando los DataFrames (si hubiera más de uno)
df = pd.concat([df_thetech], axis=0, ignore_index=True)

print("Filas y columnas:", df.shape)
df.head()

In [ ]:
# =====================================================
# 3. Definir columnas a excluir del análisis
# =====================================================
exclude_cols = ["Filename", "Name", "URL"]
columns_to_analyze = [c for c in df.columns if c not in exclude_cols]

print("Columnas que se van a analizar:\n", columns_to_analyze)

In [ ]:
# =====================================================
# 4. Función para separar valores múltiples
# =====================================================
def split_multivalue(column_series):
    """
    Convierte strings con valores separados por comas en listas de valores.
    Ejemplo: 'Abstracción, Descomposición' -> ['Abstracción', 'Descomposición']
    """
    # Reemplazamos NaN por cadena vacía para evitar errores al hacer split
    column_series = column_series.fillna("")
    
    # En cada fila se separan por comas y se limpian espacios
    splitted_lists = []
    for cell in column_series:
        # Separamos por comas
        parts = cell.split(",")
        # Eliminamos espacios sobrantes y cadenas vacías
        parts = [p.strip() for p in parts if p.strip() != ""]
        splitted_lists.append(parts)
    
    return splitted_lists

In [ ]:
# =====================================================
# 5. Análisis de la distribución de cada columna
# =====================================================
for col in columns_to_analyze:
    # Parseamos la columna (por si tiene múltiples valores)
    list_of_lists = split_multivalue(df[col])

    # Unimos todo en una sola lista para contar frecuencias
    combined = []
    for items in list_of_lists:
        combined.extend(items)

    # Creamos el contador de frecuencia
    counter = Counter(combined)
    
    # Si una columna está realmente vacía (sin datos válidos), continuamos
    if not counter:
        print(f"\n[Columna '{col}'] No tiene datos o todos están vacíos.")
        continue
    
    # Imprimimos la distribución
    print(f"\n=== Distribución de la columna '{col}' ===")
    for key, count in counter.most_common():
        print(f"  {key}: {count} recurso(s)")

    # Representamos en un gráfico de barras si hay más de un valor
    if len(counter) > 1:        
        plt.figure(figsize=(8, 4))
        plt.bar(counter.keys(), counter.values(), color="skyblue")
        plt.title(f"Distribución - {col}")
        plt.xlabel(col)
        plt.ylabel("Frecuencia")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()
    else:
        print("  (Solo existe un valor único en esta columna, sin gráfico.)")